In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn import preprocessing
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [5]:
# /home/kazim/Desktop/projects/IE490
fileName = '/home/kazim/Desktop/projects/IE490/input/tubitak_data2_processesed.csv'
df = pd.read_csv(fileName,  sep = ',')

In [6]:
#preview data
df.head()

,ilce_kod,mahalle_kod,bagimsiz_bolum_kat,yuzolcumu,yasal_burut_alani,mevcut_alani,adil_piyasa_degeri_yasal_durum,area,duration
0,85,55035,0.0,478.33,85.0,85.0,62000.0,19.765702,0
1,86,57190,1.0,7961.00,97.0,97.0,100000.0,70.000000,0
2,80,3835,-1.0,722.00,100.0,100.0,85000.0,36.000000,0
3,80,3837,-2.0,353.00,51.0,51.0,45000.0,32.000000,0
4,86,57190,1.0,667.00,97.0,97.0,110000.0,56.000000,0


In [7]:
# df.drop(df.index[df.ilce_kod!=79],inplace=True)
# df
# df.drop('ilce_kod',axis=1,inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102191 entries, 0 to 102190
Data columns (total 9 columns):
ilce_kod                          102191 non-null int64
mahalle_kod                       102191 non-null int64
bagimsiz_bolum_kat                102191 non-null float64
yuzolcumu                         102191 non-null float64
yasal_burut_alani                 102191 non-null float64
mevcut_alani                      102191 non-null float64
adil_piyasa_degeri_yasal_durum    102191 non-null float64
area                              102191 non-null float64
duration                          102191 non-null int64
dtypes: float64(6), int64(3)
memory usage: 7.0 MB


In [10]:
mahalle = df["mahalle_kod"]
ilce = df["ilce_kod"]

In [19]:
print(df['mahalle_kod'].unique().shape)
print(df['ilce_kod'].unique().shape)

(302,)
(18,)


In [20]:
#we can drop yasal burut alani as it has almost 1 correlation with mevcut alan
df = df.drop('yasal_burut_alani',axis=1)

### One Hot Encoding for Categorical Variables

In [21]:
df = pd.get_dummies(df, columns=["ilce_kod"])
df = pd.get_dummies(df, columns=["mahalle_kod"])

In [22]:
df.head()

,bagimsiz_bolum_kat,yuzolcumu,mevcut_alani,adil_piyasa_degeri_yasal_durum,area,duration,ilce_kod_64,ilce_kod_65,ilce_kod_68,ilce_kod_70,...,mahalle_kod_74562,mahalle_kod_74631,mahalle_kod_74677,mahalle_kod_74678,mahalle_kod_74704,mahalle_kod_74751,mahalle_kod_74798,mahalle_kod_74883,mahalle_kod_74885,mahalle_kod_74930
0,0.0,478.33,85.0,62000.0,19.765702,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,7961.00,97.0,100000.0,70.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-1.0,722.00,100.0,85000.0,36.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-2.0,353.00,51.0,45000.0,32.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,667.00,97.0,110000.0,56.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df.shape

(102191, 326)

## Model Training and Evaulation

### 1. Random Forest

A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and use averaging to improve the predictive accuracy and control over-fitting.

In [148]:
X = df.drop('adil_piyasa_degeri_yasal_durum',axis=1)
y = df['adil_piyasa_degeri_yasal_durum']

In [25]:
RANDOM_STATE = 42

In [26]:
from sklearn.ensemble import RandomForestRegressor

In [34]:
regr = RandomForestRegressor(bootstrap=True,
                             oob_score=True,
                             max_depth= 30,
                             max_features= 3,
                             min_samples_leaf= 1,
                             min_samples_split= 4,
                             n_estimators= 300,
                             random_state = RANDOM_STATE)
rf = regr.fit(X, y)

# 80 with outliers
#                              oob_score=True,
#                              max_depth = 30,
#                              max_features = 3,
#                              min_samples_leaf = 1,
#                              min_samples_split = 4,
#                              n_estimators = 300,
#                              random_state = RANDOM_STATE

# 79 outlier removed

# {'bootstrap': True,
#  'max_depth': 10,
#  'max_features': 5,
#  'min_samples_leaf': 1,
#  'min_samples_split': 4,
#  'n_estimators': 400}

# not removed

# {'bootstrap': True,
#  'max_depth': 20,
#  'max_features': 5,
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 300}


### Cross Validation for Parameter Optimization

In [32]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 600, num = 3)]
# Number of features to consider at every split
max_features = ['auto', 2,3,4,'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 4,8]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4,8]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf
               }
pprint(random_grid)

{'max_depth': [10, 35, 60, 85, 110, None],
 'max_features': ['auto', 2, 3, 4, 'sqrt'],
 'min_samples_leaf': [2, 4, 8],
 'min_samples_split': [2, 4, 8],
 'n_estimators': [200, 400, 600]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(random_state = RANDOM_STATE,oob_score=True)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 10, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=1, random_state=RANDOM_STATE, n_jobs=-1,                             
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X, y);

In [128]:
rf_random.best_params_

{'max_depth': 10,
 'max_features': 4,
 'min_samples_leaf': 2,
 'min_samples_split': 4,
 'n_estimators': 400}

In [129]:
# print(best_random.feature_importances_)

In [130]:
best_random = rf_random.best_estimator_

**We can now perform grid search building on the result from the random search. We will test a range of hyperparameters around the best values returned by random search.**

In [131]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 20],
    'max_features': [ 4,5,6],
    'min_samples_leaf': [1, 2,3],
    'min_samples_split': [2,4,6],
    'n_estimators': [300,400,500]
}

# Create a base model
rf = RandomForestRegressor(random_state = RANDOM_STATE,oob_score=True)
rf.fit(X,y)
 
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 1, return_train_score=True)

In [132]:
# Fit the grid search to the data
grid_search.fit(X, y);

Fitting 3 folds for each of 162 candidates, totalling 486 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 486 out of 486 | elapsed:  3.1min finished


In [133]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 4,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 300}

In [134]:
best_grid = grid_search.best_estimator_

0           85
1           86
2           80
3           80
4           86
5           80
6           86
7           83
8           64
9           70
10          85
11          83
12          86
13          73
14          83
15          83
16          83
17          83
18          85
19          70
20          85
21          80
22          80
23          85
24          86
25          85
26          86
27          80
28          80
29          65
          ... 
102161      86
102162      80
102163      87
102164      80
102165      70
102166      87
102167      80
102168    1594
102169      80
102170      80
102171      73
102172      73
102173      70
102174      73
102175      80
102176      80
102177      80
102178      65
102179      86
102180      80
102181      83
102182      70
102183      77
102184      73
102185      80
102186      73
102187      87
102188      86
102189      73
102190      70
Name: ilce_kod, Length: 102191, dtype: int64